In [106]:
# ============================================================================
# DATASET 2: ALOJAMIENTOS TURÍSTICOS - LIMPIEZA Y GENERACIÓN JSON
# ============================================================================
# Actividad 1: Limpieza de datos
# Asignatura: Bases de Datos para el Big Data
# Dataset: alojamientos_turisticos.csv
# ============================================================================

import pandas as pd
import numpy as np
import json
from datetime import datetime
import re
import os
import csv

In [107]:
# ============================================================================
# DATASET 2: ALOJAMIENTOS TURÍSTICOS - LIMPIEZA Y GENERACIÓN JSON
# ============================================================================
# Actividad 1: Limpieza de datos
# Asignatura: Bases de Datos para el Big Data
# Dataset: alojamientos_turisticos.csv
# ============================================================================

# ============================================================================
# ETAPA 1: CARGA Y EXPLORACIÓN INICIAL
# ============================================================================

def cargar_dataset(filename: str) -> pd.DataFrame | None:
    try:
        base_dir = os.getcwd()
        file_path = os.path.join(base_dir, "..", "data", filename)

        if not os.path.exists(file_path):
            raise FileNotFoundError(f"El archivo {file_path} no existe.")

        ext = os.path.splitext(filename)[1].lower()

        if ext == ".csv":
            # Intentar abrir con UTF-8, si falla usar latin-1
            for encoding in ["utf-8", "latin-1"]:
                try:
                    with open(file_path, 'r', encoding=encoding) as f:
                        sample = f.read(2048)
                        sniffer = csv.Sniffer()
                        dialect = sniffer.sniff(sample)
                        detected_sep = dialect.delimiter

                    data = pd.read_csv(file_path, encoding=encoding, sep=detected_sep)
                    break  # Si carga correctamente, salir del bucle
                except UnicodeDecodeError:
                    continue
            else:
                raise UnicodeDecodeError("No se pudo decodificar el archivo con utf-8 ni latin-1")

        elif ext in [".xls", ".xlsx"]:
            data = pd.read_excel(file_path)
        else:
            raise ValueError("Formato de archivo no soportado. Use CSV o Excel (.xls, .xlsx).")

        print(f">>> El archivo '{filename}' se cargó correctamente. Filas: {data.shape[0]}, Columnas: {data.shape[1]}")
        return data

    except FileNotFoundError as e:
        print(e)
    except pd.errors.ParserError as e:
        print(f"Error de formato: {e}")
    except ValueError as e:
        print(e)
    except Exception as e:
        print(f"Ocurrió un error inesperado: {e}")

    return None


def explorar_dataset(df):
    """
    Muestra información básica del dataset
    """
    print("\n" + "="*80)
    print("EXPLORACIÓN INICIAL DEL DATASET")
    print("="*80)
    
    print("\n1. Primeras filas:")
    display(df.head(3))
    
    print("\n2. Información de columnas:")
    print(df.info())
    
    print("\n3. Valores nulos por columna:")
    print(df.isnull().sum())
    
    print("\n4. Valores únicos en columnas clave:")
    for col in ['flightNumber', 'plane', 'dep_airport_code', 'arr_airport_code']:
        if col in df.columns:
            print(f"  - {col}: {df[col].nunique()} valores únicos")
    
    return df

In [108]:
df = cargar_dataset('alojamientos_turisticos.csv')

>>> El archivo 'alojamientos_turisticos.csv' se cargó correctamente. Filas: 10974, Columnas: 14


In [109]:

# ============================================================================
# ETAPA 2: NORMALIZACIÓN DE NOMBRES DE COLUMNAS
# ============================================================================

def normalizar_nombres_columnas(df):
    """
    Convierte nombres de columnas a snake_case sin tildes ni caracteres especiales
    """
    print("\n" + "="*80)
    print("NORMALIZACIÓN DE NOMBRES DE COLUMNAS")
    print("="*80)
    
    def a_snake_case(texto):
        # Eliminar tildes
        texto = texto.replace('á', 'a').replace('é', 'e').replace('í', 'i')
        texto = texto.replace('ó', 'o').replace('ú', 'u').replace('ñ', 'n')
        texto = texto.replace('Á', 'A').replace('É', 'E').replace('Í', 'I')
        texto = texto.replace('Ó', 'O').replace('Ú', 'U').replace('Ñ', 'N')
        
        # Convertir a minúsculas
        texto = texto.lower()
        
        # Reemplazar espacios y caracteres especiales por guión bajo
        texto = re.sub(r'[^a-z0-9]+', '_', texto)
        
        # Eliminar guiones bajos al inicio y final
        texto = texto.strip('_')
        
        return texto
    
    columnas_originales = df.columns.tolist()
    columnas_nuevas = [a_snake_case(col) for col in columnas_originales]
    
    df.columns = columnas_nuevas
    
    print(f"\n✓ Nombres de columnas normalizados")
    print(f"  Total de columnas: {len(columnas_nuevas)}")
    print("\nEjemplos de cambios:")
    for orig, nuevo in list(zip(columnas_originales, columnas_nuevas))[:5]:
        if orig != nuevo:
            print(f"  '{orig}' → '{nuevo}'")
    
    return df



In [110]:
# 3. Normalizar nombres de columnas
df = normalizar_nombres_columnas(df)



NORMALIZACIÓN DE NOMBRES DE COLUMNAS

✓ Nombres de columnas normalizados
  Total de columnas: 14

Ejemplos de cambios:


In [111]:

# ============================================================================
# ETAPA 3: LIMPIEZA DE VALORES NULOS Y MARCADORES
# ============================================================================

def limpiar_valores_nulos(df):
    """
    Convierte marcadores de nulo ("-", espacios vacíos, etc.) a NaN
    """
    print("\n" + "="*80)
    print("LIMPIEZA DE VALORES NULOS")
    print("="*80)
    
    # Reemplazar "-" y strings vacíos por NaN
    df = df.replace(['-', '', ' ', 'nan', 'NaN'], np.nan)
    
    # Limpiar espacios en blanco en columnas de texto
    for col in df.select_dtypes(include=['object']).columns:
        df[col] = df[col].apply(lambda x: x.strip() if isinstance(x, str) else x)
    
    print(f"\n✓ Valores nulos limpiados")
    print(f"\nValores nulos por columna:")
    nulos = df.isnull().sum()
    print(nulos[nulos > 0])
    
    return df



In [112]:
# 4. Limpiar valores nulos
df = limpiar_valores_nulos(df)


LIMPIEZA DE VALORES NULOS

✓ Valores nulos limpiados

Valores nulos por columna:
denominacion     2326
via_nombre         16
numero             13
bloque          10787
portal          10530
escalera         9830
planta           2136
puerta           2867
cdpostal          650
localidad           3
dtype: int64


In [113]:

# ============================================================================
# ETAPA 4: PROBLEMA 3 - NORMALIZACIÓN DE CAMPOS DE DIRECCIÓN
# ============================================================================

def normalizar_direccion(df):
    """
    Normaliza campos de dirección: via_tipo, via_nombre, planta, puerta
    """
    print("\n" + "="*80)
    print("PROBLEMA 3: NORMALIZACIÓN DE DIRECCIÓN")
    print("="*80)
    
    # Normalizar via_tipo a mayúsculas
    if 'via_tipo' in df.columns:
        df['via_tipo'] = df['via_tipo'].str.upper()
        print(f"\n✓ via_tipo normalizado a mayúsculas")
        print(f"  Valores únicos: {df['via_tipo'].unique()}")
    
    # Normalizar via_nombre: primera letra mayúscula
    if 'via_nombre' in df.columns:
        df['via_nombre'] = df['via_nombre'].apply(
            lambda x: x.title() if isinstance(x, str) else x
        )
        print(f"\n✓ via_nombre normalizado (Title Case)")
    
    # Normalizar planta: extraer solo el número
    if 'planta' in df.columns:
        def limpiar_planta(valor):
            if pd.isna(valor):
                return None
            valor_str = str(valor)
            # Extraer solo dígitos
            numeros = re.findall(r'\d+', valor_str)
            return numeros[0] if numeros else None
        
        df['planta'] = df['planta'].apply(limpiar_planta)
        print(f"\n✓ planta normalizada (solo números)")
        print(f"  Valores únicos: {df['planta'].unique()}")
    
    # Normalizar puerta: mapeo de abreviaciones
    if 'puerta' in df.columns:
        mapeo_puerta = {
            'DCHA.': 'Derecha',
            'DCHA': 'Derecha',
            'D': 'Derecha',
            'IZDA.': 'Izquierda',
            'IZDA': 'Izquierda',
            'I': 'Izquierda',
            'IZ': 'Izquierda',
            'EXT_IZ': 'Exterior Izquierda',
            'PTA. I': 'Izquierda'
        }
        
        df['puerta'] = df['puerta'].apply(
            lambda x: mapeo_puerta.get(str(x).upper(), x) if pd.notna(x) else x
        )
        print(f"\n✓ puerta normalizada")
        print(f"  Valores únicos: {df['puerta'].unique()}")
    
    return df



In [114]:
# 5. Normalizar dirección (Problema 3)
df = normalizar_direccion(df)


PROBLEMA 3: NORMALIZACIÓN DE DIRECCIÓN

✓ via_tipo normalizado a mayúsculas
  Valores únicos: ['PASEO' 'CALLE' 'CRA' 'PLAZA' 'CUSTA' 'AVDA' 'CTRA' 'AVIA' 'BULEV'
 'CSTAN' 'GTA' 'TRVA' 'COL' 'CMNO' 'RONDA' 'FINCA' 'PRAJE' 'CLLJA' 'PSAJE'
 'CLLON' 'URB' 'PZO' 'BARRO' 'SECT' 'DISEM' 'RTDA' 'POLIG' 'LUGAR' 'EXTRR']

✓ via_nombre normalizado (Title Case)

✓ planta normalizada (solo números)
  Valores únicos: [None '3' '8' '4' '2' '5' '1' '7' '9' '6' '41' '3568' '1234' '0' '20' '14'
 '13' '10' '11' '25' '24' '19' '16' '12' '15' '22' '00' '01' '02' '102'
 '104' '110' '111' '208' '311' '402' '405' '406' '408' '410' '501' '602'
 '610' '702' '705' '706' '709' '710' '801' '802' '803' '804' '805' '806'
 '807' '401' '210' '106' '223' '219' '274' '04']

✓ puerta normalizada
  Valores únicos: [nan 'Derecha' '2' 'Exterior Izquierda' 'A' 'Izquierda' 'C' '3' 'B' '1'
 'IZQ.' 'IZ DCH' 'A y B' '5 y 6' 'CENTRO' 'B y C' 'IZQDA' 'DHA' 'H' 'F'
 'DRCHA' 'A C D' 'PTA. D' 'EXT. I' 'Ext.' 'EXT' 'DHCA' 'PTA. 1' 'L

In [115]:

# ============================================================================
# ETAPA 5: PROBLEMA 4 - SEPARACIÓN DE TIPO Y CATEGORÍA
# ============================================================================

def separar_tipo_categoria(df):
    """
    Separa el campo 'categoria' en 'categoria_nivel' (numérico) y 'categoria_tipo' (texto)
    Normaliza 'alojamiento_tipo'
    """
    print("\n" + "="*80)
    print("PROBLEMA 4: SEPARACIÓN DE TIPO Y CATEGORÍA")
    print("="*80)
    
    if 'categoria' in df.columns:
        # Extraer nivel (número) y tipo (texto) de 'categoria'
        # Ejemplo: "3-HOTEL" → nivel=3, tipo="HOTEL"
        
        def extraer_nivel(valor):
            if pd.isna(valor):
                return None
            match = re.match(r'^(\d+)', str(valor))
            return int(match.group(1)) if match else None
        
        def extraer_tipo(valor):
            if pd.isna(valor):
                return None
            match = re.search(r'-(.+)$', str(valor))
            return match.group(1).strip() if match else str(valor)
        
        df['categoria_nivel'] = df['categoria'].apply(extraer_nivel)
        df['categoria_tipo'] = df['categoria'].apply(extraer_tipo)
        
        print(f"\n✓ Campo 'categoria' separado en:")
        print(f"  - categoria_nivel (numérico): {df['categoria_nivel'].unique()}")
        print(f"  - categoria_tipo (texto): {df['categoria_tipo'].unique()}")
    
    # Normalizar alojamiento_tipo a mayúsculas
    if 'alojamiento_tipo' in df.columns:
        df['alojamiento_tipo'] = df['alojamiento_tipo'].str.upper()
        print(f"\n✓ alojamiento_tipo normalizado")
        print(f"  Valores únicos: {df['alojamiento_tipo'].unique()}")
    
    return df



In [116]:
# 6. Separar tipo y categoría (Problema 4)
df = separar_tipo_categoria(df)


PROBLEMA 4: SEPARACIÓN DE TIPO Y CATEGORÍA

✓ Campo 'categoria' separado en:
  - categoria_nivel (numérico): [ 3.  2. nan  1.  4.  5.]
  - categoria_tipo (texto): ['HOTEL' 'PENSION' 'HOSTAL' 'CASA HUESPEDES' '5 estrellas L'
 'HOTEL-APART.' 'APARTAMENTO RURAL' 'HOJA DE ROBLE' 'CASA RURAL'
 'APART-TURISTICO' 'SIN CATEGORIA' 'CAMPING']

✓ alojamiento_tipo normalizado
  Valores únicos: ['HOTEL' 'PENSION' 'HOSTAL' 'CASA HUESPEDES' 'HOTEL-APART.'
 'APARTAMENTO RURAL' 'HOTEL RURAL' 'CASA RURAL' 'APART-TURISTICO'
 'HOSTERIAS' 'VIVIENDAS DE USO TU' 'CAMPING']


In [117]:

# ============================================================================
# ETAPA 6: VALIDACIÓN Y RESUMEN FINAL
# ============================================================================

def validar_limpieza(df):
    """
    Valida que la limpieza se haya realizado correctamente
    """
    print("\n" + "="*80)
    print("VALIDACIÓN Y RESUMEN FINAL")
    print("="*80)
    
    print(f"\n--- Dimensiones finales ---")
    print(f"  Registros: {len(df)}")
    print(f"  Columnas: {len(df.columns)}")
    
    print(f"\n--- Tipos de datos ---")
    print(df.dtypes)
    
    print(f"\n--- Valores nulos totales ---")
    total_nulos = df.isnull().sum().sum()
    total_valores = df.shape[0] * df.shape[1]
    porcentaje_nulos = (total_nulos / total_valores) * 100
    print(f"  Total: {total_nulos} ({porcentaje_nulos:.2f}%)")
    
    print(f"\n--- Primeras 3 filas del dataset limpio ---")
    print(df.head(3))
    
    return df



In [118]:
# 7. Validar limpieza
df_final = validar_limpieza(df)


VALIDACIÓN Y RESUMEN FINAL

--- Dimensiones finales ---
  Registros: 10974
  Columnas: 16

--- Tipos de datos ---
alojamiento_tipo     object
categoria            object
denominacion         object
via_tipo             object
via_nombre           object
numero               object
bloque               object
portal               object
escalera             object
planta               object
puerta               object
cdpostal            float64
localidad            object
signatura            object
categoria_nivel     float64
categoria_tipo       object
dtype: object

--- Valores nulos totales ---
  Total: 50416 (28.71%)

--- Primeras 3 filas del dataset limpio ---
  alojamiento_tipo  categoria  denominacion via_tipo       via_nombre numero  \
0            HOTEL    3-HOTEL  GRAN LEGAZPI    PASEO    De La Chopera     71   
1          PENSION  2-PENSION        ISABEL    CALLE      De La Salud     13   
2           HOSTAL   2-HOSTAL        BESAYA    CALLE  De San Bernardo     13   

  

In [119]:

# ============================================================================
# ETAPA 7: EXPORTACIÓN A JSON
# ============================================================================

def safe_value(v):
    """
    Convierte valores NaN de pandas a None (que será null en JSON).
    Convierte Timestamp a string ISO 8601.
    Convierte numpy types a tipos nativos de Python.
    """
    if pd.isna(v):
        return None
    
    # Convertir Timestamp de pandas a string ISO 8601
    if isinstance(v, pd.Timestamp):
        return v.strftime('%Y-%m-%dT%H:%M:%S')
    
    # Convertir numpy types a tipos nativos de Python
    if isinstance(v, (np.integer, )):
        return int(v)
    if isinstance(v, (np.floating, )):
        return float(v)
    
    # Convertir datetime de Python a string ISO 8601
    if isinstance(v, datetime):
        return v.strftime('%Y-%m-%dT%H:%M:%S')
    
    return v


def transformar_a_json_estructura(df):
    """
    Transforma el DataFrame limpio a estructura JSON para alojamientos
    
    Returns:
        Lista de diccionarios con la estructura JSON de alojamientos
    """
    alojamientos_json = []
    ts = datetime.now().strftime('%Y-%m-%dT%H:%M:%S')
    
    for idx, row in df.iterrows():
        # Crear documento base
        alojamiento = {
            "alojamiento_id": int(idx) + 1,  # ID secuencial
            "timestamp": ts,
            "datos": {}
        }
        
        # Añadir todos los campos del DataFrame
        for col in df.columns:
            alojamiento["datos"][col] = safe_value(row[col])
        
        alojamientos_json.append(alojamiento)
    
    return alojamientos_json


def exportar_json(alojamientos_json, ruta_salida='dataset2_alojamientos_limpio.json', 
                  num_documentos=None, indent=2):
    """
    Exporta la lista de alojamientos a un archivo JSON
    
    Args:
        alojamientos_json: lista de diccionarios con estructura de alojamientos
        ruta_salida: ruta del archivo JSON de salida
        num_documentos: número de documentos a exportar. Si es None, exporta todos.
        indent: nivel de indentación para legibilidad
    """
    # Determinar cuántos documentos exportar
    if num_documentos is None:
        documentos_a_exportar = alojamientos_json
        total_exportado = len(alojamientos_json)
    else:
        documentos_a_exportar = alojamientos_json[:num_documentos]
        total_exportado = min(num_documentos, len(alojamientos_json))
    
    # Exportar a JSON
    with open(ruta_salida, 'w', encoding='utf-8') as f:
        json.dump(documentos_a_exportar, f, ensure_ascii=False, indent=indent)
    
    print(f"\n✓ Archivo JSON exportado: {ruta_salida}")
    print(f"  Total de alojamientos exportados: {total_exportado}")
    print(f"  Total de alojamientos disponibles: {len(alojamientos_json)}")
    
    if num_documentos and num_documentos < len(alojamientos_json):
        print(f"  ⚠ Se exportaron solo los primeros {num_documentos} documentos")
    
    tamanio_kb = round(len(json.dumps(documentos_a_exportar, ensure_ascii=False)) / 1024, 2)
    print(f"  Tamaño del archivo: {tamanio_kb} KB")


def mostrar_ejemplo_json(alojamientos_json, num_ejemplos=2):
    """
    Muestra ejemplos del JSON generado
    """
    print("\n" + "="*80)
    print(f"EJEMPLOS DE DOCUMENTOS JSON GENERADOS (primeros {num_ejemplos})")
    print("="*80)
    
    for i, alojamiento in enumerate(alojamientos_json[:num_ejemplos], 1):
        print(f"\n--- Alojamiento {i} ---")
        print(json.dumps(alojamiento, ensure_ascii=False, indent=2))

In [120]:
# 8. Transformar a JSON
alojamientos_json = transformar_a_json_estructura(df_final)
mostrar_ejemplo_json(alojamientos_json, num_ejemplos=2)


EJEMPLOS DE DOCUMENTOS JSON GENERADOS (primeros 2)

--- Alojamiento 1 ---
{
  "alojamiento_id": 1,
  "timestamp": "2025-11-28T01:11:48",
  "datos": {
    "alojamiento_tipo": "HOTEL",
    "categoria": "3-HOTEL",
    "denominacion": "GRAN LEGAZPI",
    "via_tipo": "PASEO",
    "via_nombre": "De La Chopera",
    "numero": "71",
    "bloque": null,
    "portal": null,
    "escalera": null,
    "planta": null,
    "puerta": null,
    "cdpostal": 28045.0,
    "localidad": "Madrid",
    "signatura": "HM-127",
    "categoria_nivel": 3.0,
    "categoria_tipo": "HOTEL"
  }
}

--- Alojamiento 2 ---
{
  "alojamiento_id": 2,
  "timestamp": "2025-11-28T01:11:48",
  "datos": {
    "alojamiento_tipo": "PENSION",
    "categoria": "2-PENSION",
    "denominacion": "ISABEL",
    "via_tipo": "CALLE",
    "via_nombre": "De La Salud",
    "numero": "13",
    "bloque": null,
    "portal": null,
    "escalera": null,
    "planta": "3",
    "puerta": null,
    "cdpostal": 28013.0,
    "localidad": "Madrid",
  

In [121]:
# 9. Exportar JSON completo
#exportar_json(alojamientos_json, ruta_salida='dataset2_alojamientos_limpio.json')
 
# 10. (Opcional) Exportar solo una muestra para pruebas
exportar_json(alojamientos_json, ruta_salida='dataset2_alojamientos_muestra_100.json', num_documentos=100)
 
print("\n" + "="*80)
print("PROCESO COMPLETADO")
print("="*80)


✓ Archivo JSON exportado: dataset2_alojamientos_muestra_100.json
  Total de alojamientos exportados: 100
  Total de alojamientos disponibles: 10974
  ⚠ Se exportaron solo los primeros 100 documentos
  Tamaño del archivo: 41.72 KB

PROCESO COMPLETADO
